# Milestone 6: 使用者註冊系統 - 專案需求規格書

## 📋 專案概述

本專案要求學生建立一個**安全的使用者註冊與認證系統**，重點在於：
- ✅ **例外處理機制**: 建立完整的自訂例外階層
- ✅ **安全驗證**: 實作輸入驗證與密碼安全機制
- ✅ **日誌系統**: 建立專業級的操作記錄
- ✅ **錯誤恢復**: 確保系統在各種錯誤情況下的穩定性

**整合章節**: Ch20 例外處理、Ch21 自訂例外、Ch22 除錯技術

## 🔧 技術需求

### 1. 自訂例外類別架構

必須建立完整的例外階層，包含以下類別：

In [ ]:
# 例外類別架構範例
from datetime import datetime

class UserSystemError(Exception):
    """系統基礎例外 - 所有自訂例外的父類別
    
    Attributes:
        message (str): 錯誤訊息
        error_code (str): 錯誤代碼
        details (dict): 詳細錯誤資訊
        timestamp (datetime): 錯誤發生時間
    """
    def __init__(self, message, error_code=None, details=None):
        super().__init__(message)
        self.message = message
        self.error_code = error_code
        self.details = details or {}
        self.timestamp = datetime.now()

    def __str__(self):
        return f"[{self.error_code}] {self.message}"

# 具體例外類別
class ValidationError(UserSystemError):
    """資料驗證例外"""
    pass

class AuthenticationError(UserSystemError):
    """身份驗證例外"""
    pass

class SecurityError(UserSystemError):
    """安全性例外"""
    pass

class DatabaseError(UserSystemError):
    """資料操作例外"""
    pass

**評分標準**:
- 📊 例外階層完整性 (4分)
- 📊 例外資訊豐富度 (3分)
- 📊 繼承關係正確性 (3分)

### 2. 輸入驗證系統

#### 2.1 使用者名稱驗證

**驗證規則**:
- ✅ 長度: 3-20 個字元
- ✅ 字元: 只允許英文字母、數字、底線 `[a-zA-Z0-9_]`
- ✅ 不能為空
- ✅ 必須唯一 (不能重複)

In [ ]:
# 使用者名稱驗證範例
import re

def validate_username(username):
    """驗證使用者名稱
    
    Args:
        username (str): 使用者名稱
        
    Raises:
        ValidationError: 當使用者名稱不符合規則時
    """
    if not username:
        raise ValidationError(
            "使用者名稱不能為空",
            error_code="EMPTY_USERNAME"
        )
    
    if len(username) < 3:
        raise ValidationError(
            "使用者名稱至少需要 3 個字元",
            error_code="USERNAME_TOO_SHORT",
            details={"min_length": 3, "current_length": len(username)}
        )
    
    if len(username) > 20:
        raise ValidationError(
            "使用者名稱不能超過 20 個字元",
            error_code="USERNAME_TOO_LONG",
            details={"max_length": 20, "current_length": len(username)}
        )
    
    if not re.match(r'^[a-zA-Z0-9_]+$', username):
        raise ValidationError(
            "使用者名稱只能包含英文字母、數字和底線",
            error_code="INVALID_USERNAME_CHARS"
        )

# 測試案例
valid_usernames = ["user123", "test_user", "UserName", "a1b2c3"]
invalid_usernames = [
    "",          # 空字串
    "ab",        # 太短
    "a" * 21,    # 太長
    "user@name", # 特殊字元
    "user name", # 空格
    "用戶",      # 中文
]

#### 2.2 密碼強度驗證

**安全要求**:
- ✅ 最少 8 個字元
- ✅ 至少包含 3 種字元類型：
  - 大寫字母 (A-Z)
  - 小寫字母 (a-z) 
  - 數字 (0-9)
  - 特殊符號 (!@#$%^&*()_+-=[]{}|;:,.<>?)
- ✅ 不能是常見弱密碼

In [ ]:
# 密碼強度驗證範例

COMMON_PASSWORDS = {
    "123456", "password", "123456789", "12345678", "12345",
    "1234567", "1234567890", "qwerty", "abc123", "Password"
}

def validate_password(password):
    """驗證密碼強度
    
    Args:
        password (str): 密碼
        
    Raises:
        SecurityError: 當密碼不符合安全要求時
    """
    if not password:
        raise SecurityError(
            "密碼不能為空",
            error_code="EMPTY_PASSWORD"
        )
    
    if len(password) < 8:
        raise SecurityError(
            "密碼至少需要 8 個字元",
            error_code="PASSWORD_TOO_SHORT",
            details={"min_length": 8, "current_length": len(password)}
        )
    
    if password.lower() in COMMON_PASSWORDS:
        raise SecurityError(
            "密碼過於簡單，請使用更安全的密碼",
            error_code="WEAK_PASSWORD"
        )
    
    # 檢查密碼複雜度
    has_upper = any(c.isupper() for c in password)
    has_lower = any(c.islower() for c in password)
    has_digit = any(c.isdigit() for c in password)
    has_special = any(c in "!@#$%^&*()_+-=[]{}|;:,.<>?" for c in password)
    
    complexity_score = sum([has_upper, has_lower, has_digit, has_special])
    
    if complexity_score < 3:
        raise SecurityError(
            "密碼需要包含至少三種類型：大寫字母、小寫字母、數字、特殊符號",
            error_code="INSUFFICIENT_COMPLEXITY",
            details={
                "has_upper": has_upper,
                "has_lower": has_lower,
                "has_digit": has_digit,
                "has_special": has_special,
                "score": complexity_score
            }
        )

# 測試案例
strong_passwords = [
    "MySecure123!",
    "P@ssw0rd2024",
    "Str0ng#Pass"
]

weak_passwords = [
    "123456",      # 常見密碼
    "password",    # 常見密碼
    "short",       # 太短
    "nouppercase", # 無大寫
    "NOLOWERCASE", # 無小寫
    "NoNumbers!",  # 無數字
    "NoSpecial123" # 無特殊字元
]

**密碼儲存**:
- 🔒 使用 PBKDF2 + SHA256 雜湊
- 🔒 每個密碼使用唯一的 salt
- 🔒 迭代次數: 100,000 次

In [ ]:
# 密碼加密範例
import hashlib
import secrets

def hash_password(password):
    """加密密碼
    
    Args:
        password (str): 原始密碼
        
    Returns:
        tuple: (salt, hashed_password)
    """
    salt = secrets.token_hex(32)
    password_hash = hashlib.pbkdf2_hmac(
        'sha256',
        password.encode('utf-8'),
        salt.encode('utf-8'),
        100000  # 迭代次數
    )
    return salt, password_hash.hex()

def verify_password(password, salt, stored_hash):
    """驗證密碼
    
    Args:
        password (str): 輸入的密碼
        salt (str): 密碼鹽值
        stored_hash (str): 儲存的密碼雜湊
        
    Returns:
        bool: 密碼是否正確
    """
    password_hash = hashlib.pbkdf2_hmac(
        'sha256',
        password.encode('utf-8'),
        salt.encode('utf-8'),
        100000
    )
    return password_hash.hex() == stored_hash

# 測試密碼加密
password = "MySecure123!"
salt, hashed = hash_password(password)
print(f"Salt 長度: {len(salt)}")
print(f"驗證結果: {verify_password(password, salt, hashed)}")
print(f"錯誤密碼: {verify_password('wrong', salt, hashed)}")

#### 2.3 電子郵件驗證

**格式要求**:
- ✅ 符合標準電子郵件格式
- ✅ 正規表達式: `^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$`
- ✅ 不能為空
- ✅ 必須唯一 (一個信箱只能註冊一個帳號)

In [ ]:
# 電子郵件驗證範例
import re

def validate_email(email):
    """驗證電子郵件格式
    
    Args:
        email (str): 電子郵件地址
        
    Raises:
        ValidationError: 當郵件格式無效時
    """
    if not email:
        raise ValidationError(
            "電子郵件不能為空",
            error_code="EMPTY_EMAIL"
        )
    
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(email_pattern, email):
        raise ValidationError(
            "電子郵件格式無效",
            error_code="INVALID_EMAIL_FORMAT",
            details={"email": email}
        )

# 測試案例
valid_emails = [
    "user@example.com",
    "test.user+tag@domain.co.uk",
    "user123@test-domain.com"
]

invalid_emails = [
    "plainaddress",
    "@missingdomain.com",
    "missing@.com",
    "spaces @domain.com"
]

# 測試驗證
for email in valid_emails:
    try:
        validate_email(email)
        print(f"✓ {email} - 有效")
    except ValidationError as e:
        print(f"❌ {email} - 無效: {e}")

for email in invalid_emails:
    try:
        validate_email(email)
        print(f"✓ {email} - 有效")
    except ValidationError as e:
        print(f"❌ {email} - 無效: {e}")

### 3. 安全機制要求

#### 3.1 登入嘗試限制

**防暴力破解機制**:
- ⚡ 最大嘗試次數: 5 次
- ⚡ 鎖定時間: 5 分鐘 (300 秒)
- ⚡ 記錄失敗嘗試的時間戳記
- ⚡ 成功登入後重置失敗計數

In [ ]:
# 安全管理範例
from datetime import datetime, timedelta

class SecurityManager:
    """安全管理類別"""
    
    def __init__(self):
        self.failed_attempts = {}  # {username: {count, locked_until}}
        self.max_attempts = 5
        self.lockout_time = 300  # 秒
    
    def check_login_attempts(self, username):
        """檢查登入嘗試次數
        
        Args:
            username (str): 使用者名稱
            
        Raises:
            SecurityError: 當嘗試次數過多時
        """
        if username in self.failed_attempts:
            attempt_info = self.failed_attempts[username]
            
            # 檢查是否仍在鎖定期間
            if attempt_info.get('locked_until') and attempt_info['locked_until'] > datetime.now():
                remaining_time = attempt_info['locked_until'] - datetime.now()
                raise SecurityError(
                    f"帳號已被鎖定，請在 {remaining_time.seconds} 秒後再試",
                    error_code="ACCOUNT_LOCKED",
                    details={
                        "locked_until": attempt_info['locked_until'],
                        "remaining_seconds": remaining_time.seconds
                    }
                )
            
            # 鎖定期已過，重置嘗試次數
            if attempt_info.get('locked_until') and attempt_info['locked_until'] <= datetime.now():
                self.failed_attempts[username] = {
                    'count': 0,
                    'locked_until': None
                }
    
    def record_failed_attempt(self, username):
        """記錄失敗嘗試
        
        Args:
            username (str): 使用者名稱
        """
        if username not in self.failed_attempts:
            self.failed_attempts[username] = {
                'count': 0,
                'locked_until': None
            }
        
        self.failed_attempts[username]['count'] += 1
        
        if self.failed_attempts[username]['count'] >= self.max_attempts:
            self.failed_attempts[username]['locked_until'] = (
                datetime.now() + timedelta(seconds=self.lockout_time)
            )
    
    def reset_failed_attempts(self, username):
        """重置失敗嘗試記錄
        
        Args:
            username (str): 使用者名稱
        """
        if username in self.failed_attempts:
            del self.failed_attempts[username]

# 測試安全管理
security = SecurityManager()

# 模擬失敗嘗試
username = "testuser"
for i in range(6):
    try:
        security.check_login_attempts(username)
        security.record_failed_attempt(username)
        print(f"嘗試 {i+1}: 記錄失敗")
    except SecurityError as e:
        print(f"嘗試 {i+1}: {e}")

#### 3.2 會話管理

**會話要求**:
- 🎫 使用安全的會話 ID (32 位元 URL-safe token)
- 🎫 記錄登入時間
- 🎫 支援登出功能
- 🎫 防止重複登入 (可選)

In [ ]:
# 會話管理範例
from dataclasses import dataclass
from datetime import datetime
import secrets

@dataclass
class LoginSession:
    """登入會話模型"""
    username: str
    login_time: datetime
    session_id: str
    is_active: bool = True

class SessionManager:
    """會話管理類別"""
    
    def __init__(self):
        self.sessions = {}  # session_id -> LoginSession
        self.current_session = None
    
    def create_session(self, username):
        """建立新會話
        
        Args:
            username (str): 使用者名稱
            
        Returns:
            str: 會話ID
        """
        session_id = secrets.token_urlsafe(32)
        session = LoginSession(
            username=username,
            login_time=datetime.now(),
            session_id=session_id
        )
        
        self.sessions[session_id] = session
        self.current_session = session
        return session_id
    
    def get_session(self, session_id):
        """取得會話資訊
        
        Args:
            session_id (str): 會話ID
            
        Returns:
            LoginSession: 會話物件
        """
        return self.sessions.get(session_id)
    
    def end_session(self, session_id):
        """結束會話
        
        Args:
            session_id (str): 會話ID
        """
        if session_id in self.sessions:
            self.sessions[session_id].is_active = False
            if self.current_session and self.current_session.session_id == session_id:
                self.current_session = None
    
    def is_logged_in(self):
        """檢查是否已登入
        
        Returns:
            bool: 是否已登入
        """
        return self.current_session is not None and self.current_session.is_active

# 測試會話管理
session_manager = SessionManager()

# 建立會話
session_id = session_manager.create_session("testuser")
print(f"會話已建立: {session_id[:10]}...")
print(f"已登入: {session_manager.is_logged_in()}")

# 結束會話
session_manager.end_session(session_id)
print(f"會話已結束")
print(f"已登入: {session_manager.is_logged_in()}")

### 4. 日誌系統需求

#### 4.1 日誌配置

**多層級日誌**:
- 📝 **DEBUG**: 詳細的除錯資訊
- 📝 **INFO**: 一般操作資訊
- 📝 **WARNING**: 警告訊息
- 📝 **ERROR**: 錯誤訊息

**輸出目標**:
- 📄 檔案日誌: `user_system.log` (詳細格式)
- 💻 控制台: 簡化格式

In [ ]:
# 日誌系統範例
import logging

class UserSystemLogger:
    """使用者系統日誌管理類別"""
    
    def __init__(self, name="UserSystem"):
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.DEBUG)
        
        # 避免重複添加 handler
        if not self.logger.handlers:
            self._setup_handlers()
    
    def _setup_handlers(self):
        """設定日誌處理器"""
        # 檔案處理器 - 詳細日誌
        file_handler = logging.FileHandler(
            'user_system.log',
            encoding='utf-8'
        )
        file_handler.setLevel(logging.DEBUG)
        file_format = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
        )
        file_handler.setFormatter(file_format)
        
        # 控制台處理器 - 簡化輸出
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_format = logging.Formatter(
            '%(levelname)s - %(message)s'
        )
        console_handler.setFormatter(console_format)
        
        self.logger.addHandler(file_handler)
        self.logger.addHandler(console_handler)
    
    def info(self, message, **kwargs):
        """記錄資訊級別日誌"""
        self.logger.info(message, extra=kwargs)
    
    def warning(self, message, **kwargs):
        """記錄警告級別日誌"""
        self.logger.warning(message, extra=kwargs)
    
    def error(self, message, **kwargs):
        """記錄錯誤級別日誌"""
        self.logger.error(message, extra=kwargs)
    
    def debug(self, message, **kwargs):
        """記錄除錯級別日誌"""
        self.logger.debug(message, extra=kwargs)

# 測試日誌系統
logger = UserSystemLogger("TestLogger")

logger.info("系統啟動")
logger.warning("登入失敗警告")
logger.error("嚴重錯誤發生")
logger.debug("除錯資訊")

#### 4.2 日誌內容要求

**必須記錄的事件**:

| 事件類型 | 日誌層級 | 記錄內容 |
|:---------|:---------|:---------|
| 系統啟動 | INFO | 系統啟動時間 |
| 使用者註冊 | INFO | 使用者名稱、註冊時間 |
| 註冊失敗 | ERROR | 失敗原因、錯誤詳情 |
| 登入成功 | INFO | 使用者名稱、登入時間、會話ID |
| 登入失敗 | WARNING | 使用者名稱、失敗原因 |
| 帳號鎖定 | WARNING | 使用者名稱、鎖定時間 |
| 密碼修改 | INFO | 使用者名稱、修改時間 |
| 登出 | INFO | 使用者名稱、登出時間 |
| 系統錯誤 | ERROR | 錯誤類型、堆疊追蹤 |
| 系統關閉 | INFO | 關閉時間 |

---

## 💻 功能需求

### 5. 核心功能規格

#### 5.1 使用者註冊

**輸入欄位**:
- 使用者名稱 (必填)
- 電子郵件 (必填)
- 密碼 (必填)
- 確認密碼 (必填)

**驗證流程**:
1. ✅ 驗證所有欄位格式
2. ✅ 檢查密碼一致性
3. ✅ 檢查使用者名稱唯一性
4. ✅ 檢查電子郵件唯一性
5. ✅ 加密密碼
6. ✅ 儲存使用者資料
7. ✅ 記錄成功日誌

**錯誤處理**:
- 任何驗證失敗都要拋出適當的例外
- 錯誤訊息要清楚指出問題所在
- 記錄詳細的錯誤日誌

#### 5.2 使用者登入

**輸入欄位**:
- 使用者名稱 (必填)
- 密碼 (必填)

**驗證流程**:
1. ✅ 檢查登入嘗試次數限制
2. ✅ 驗證使用者是否存在
3. ✅ 檢查帳號是否啟用
4. ✅ 驗證密碼正確性
5. ✅ 重置失敗嘗試記錄
6. ✅ 建立登入會話
7. ✅ 更新最後登入時間
8. ✅ 記錄成功日誌

**安全考量**:
- 不論使用者名稱是否存在，密碼錯誤時都要顯示相同訊息
- 記錄所有失敗嘗試
- 實作帳號鎖定機制

#### 5.3 個人資料管理

**查看個人資料**:
- 顯示使用者名稱
- 顯示電子郵件
- 顯示註冊時間
- 顯示最後登入時間
- 顯示登入次數 (可選)

**修改個人資料**:
- 允許修改電子郵件 (需驗證唯一性)
- 允許修改個人簡介 (可選欄位)
- 不允許修改使用者名稱 (安全考量)
- 所有修改都要記錄日誌

#### 5.4 密碼管理

**修改密碼流程**:
1. ✅ 驗證當前密碼
2. ✅ 驗證新密碼強度
3. ✅ 確認新密碼一致性
4. ✅ 檢查新密碼不等於舊密碼
5. ✅ 加密新密碼
6. ✅ 更新密碼資料
7. ✅ 記錄密碼修改日誌

**安全要求**:
- 舊密碼驗證失敗要記錄警告日誌
- 新密碼必須符合強度要求
- 密碼修改成功後建議重新登入 (可選)

#### 5.5 會話管理

**登入狀態管理**:
- 追蹤當前登入使用者
- 維護會話 ID 映射
- 支援會話驗證

**登出功能**:
- 清除當前會話
- 更新會話狀態為非活躍
- 記錄登出日誌
- 清理敏感資料

---

## 🎯 介面需求

### 6. 使用者介面規格

#### 6.1 主選單 (未登入)

```
==================================================
  🔐 使用者註冊系統 v2.0
==================================================
  目前狀態：未登入
--------------------------------------------------
1. 使用者註冊
2. 使用者登入
3. 系統資訊
0. 結束程式
==================================================
請選擇功能 (0-3): 
```

#### 6.2 會員選單 (已登入)

```
==================================================
  🔐 使用者註冊系統 v2.0
==================================================
  目前登入：使用者名稱
--------------------------------------------------
1. 查看個人資料
2. 修改個人資料
3. 修改密碼
4. 查看登入記錄
5. 登出
0. 結束程式
==================================================
請選擇功能 (0-5): 
```

#### 6.3 錯誤訊息格式

**成功訊息**:
```
✓ 操作成功訊息
```

**錯誤訊息**:
```
❌ [錯誤代碼] 錯誤描述
💡 建議解決方案 (可選)
```

**警告訊息**:
```
⚠️ 警告訊息
```

**資訊訊息**:
```
ℹ️ 資訊內容
```

---

## 📊 評分標準詳細說明

### 7. 評分細則

#### 7.1 例外處理機制 (30分)

| 評分項目 | 滿分 | 評分細則 |
|:---------|:-----|:---------|
| **自訂例外類別** | 10分 | |
| - 例外階層完整性 | 4分 | 建立 UserSystemError 基礎類別及 4 個子類別 |
| - 例外資訊豐富度 | 3分 | 包含 message, error_code, details, timestamp |
| - 繼承關係正確性 | 3分 | 正確使用繼承，覆寫 `__init__` 和 `__str__` |
| **try/except 使用** | 10分 | |
| - 例外捕捉適當性 | 5分 | 捕捉特定例外類型，不使用裸露的 except |
| - 例外處理邏輯 | 5分 | 有適當的錯誤恢復機制或使用者提示 |
| **錯誤訊息設計** | 10分 | |
| - 訊息清晰易懂 | 5分 | 錯誤訊息對使用者友善，指出具體問題 |
| - 錯誤代碼完整 | 5分 | 所有例外都有對應的錯誤代碼 |

#### 7.2 安全驗證機制 (25分)

| 評分項目 | 滿分 | 評分細則 |
|:---------|:-----|:---------|
| **輸入驗證** | 10分 | |
| - 使用者名稱驗證 | 3分 | 長度、字元、唯一性檢查完整 |
| - 電子郵件驗證 | 3分 | 格式驗證與唯一性檢查 |
| - 資料完整性驗證 | 4分 | 空值檢查、型態驗證等 |
| **密碼安全** | 10分 | |
| - 密碼強度檢查 | 5分 | 長度、複雜度、常見密碼檢查 |
| - 密碼儲存安全 | 5分 | 使用 PBKDF2+SHA256，唯一 salt |
| **登入保護** | 5分 | |
| - 嘗試次數限制 | 3分 | 5次失敗後鎖定帳號 |
| - 帳號鎖定機制 | 2分 | 時間戳記管理，鎖定時間 5分鐘 |

#### 7.3 日誌系統 (20分)

| 評分項目 | 滿分 | 評分細則 |
|:---------|:-----|:---------|
| **日誌配置** | 10分 | |
| - 多層級設定 | 5分 | DEBUG, INFO, WARNING, ERROR 層級設定 |
| - 輸出格式化 | 5分 | 檔案與控制台不同格式，時間戳記完整 |
| **日誌內容** | 10分 | |
| - 操作記錄完整 | 5分 | 註冊、登入、登出、修改等操作都有記錄 |
| - 錯誤追蹤詳細 | 5分 | 例外發生時記錄完整堆疊資訊 |

#### 7.4 系統功能 (15分)

| 評分項目 | 滿分 | 評分細則 |
|:---------|:-----|:---------|
| **註冊功能** | 5分 | 完整的使用者註冊流程，包含驗證與儲存 |
| **登入功能** | 5分 | 安全的身份驗證，會話建立 |
| **會話管理** | 5分 | 登入狀態追蹤，安全登出 |

#### 7.5 程式品質 (10分)

| 評分項目 | 滿分 | 評分細則 |
|:---------|:-----|:---------|
| **程式架構** | 5分 | |
| - 模組化設計 | 3分 | 功能分離清楚，類別職責單一 |
| - 類別設計合理 | 2分 | 使用合適的設計模式，程式碼可讀性高 |
| **註解文件** | 5分 | |
| - docstring 完整 | 3分 | 所有類別與函式都有完整的說明文件 |
| - 註解適當 | 2分 | 關鍵邏輯有清楚的註解說明 |

---

## 🎯 加分項目

### 8. 進階功能 (額外加分)

#### 8.1 進階安全功能 (+5分)
- 🔐 密碼歷史檢查 (不允許重複使用近期密碼)
- 🔐 會話過期管理 (閒置自動登出)
- 🔐 IP 位址記錄與檢查
- 🔐 雙因子認證 (TOTP)

#### 8.2 進階日誌功能 (+5分)
- 📊 日誌分析功能 (統計登入頻率、失敗模式)
- 📊 安全事件報告生成
- 📊 效能監控 (回應時間記錄)
- 📊 自動日誌輪替與壓縮

#### 8.3 使用者體驗優化 (+3分)
- 🎨 彩色輸出與格式化
- 🎨 進度提示與載入動畫
- 🎨 互動式密碼輸入 (隱藏密碼顯示)
- 🎨 多語言支援

#### 8.4 資料持久化 (+5分)
- 💾 SQLite 資料庫整合
- 💾 JSON 檔案儲存
- 💾 資料備份與還原功能
- 💾 資料遷移腳本

#### 8.5 測試覆蓋率 (+5分)
- 🧪 單元測試覆蓋率 > 80%
- 🧪 整合測試完整
- 🧪 效能測試
- 🧪 安全性測試

---

## ✅ 提交要求

### 9. 交付清單

#### 9.1 必要檔案
- 📁 `user_registration_system.py` - 主程式檔案
- 📁 `requirements.txt` - 相依套件清單 (如果有)
- 📁 `README.md` - 專案說明文件

#### 9.2 可選檔案
- 📁 `tests/` - 測試檔案目錄
- 📁 `docs/` - 額外文件
- 📁 `config/` - 設定檔案

#### 9.3 程式碼要求
- ✅ 符合 PEP 8 編碼規範
- ✅ 所有函式都有 docstring
- ✅ 適當的錯誤處理
- ✅ 程式碼註解充足
- ✅ 變數命名語意化

#### 9.4 測試要求
- ✅ 所有功能都要經過測試
- ✅ 邊界條件測試
- ✅ 異常情況處理測試
- ✅ 安全機制驗證

#### 9.5 文件要求
- ✅ 安裝與執行說明
- ✅ 功能使用指南
- ✅ 已知問題與限制
- ✅ 未來改進建議

---

## 🏆 成功標準

### 10. 專案完成指標

#### 10.1 功能完整性
- ✅ 所有核心功能都能正常運作
- ✅ 使用者能完成完整的註冊→登入→使用→登出流程
- ✅ 錯誤情況都有適當的處理與提示

#### 10.2 安全性達標
- ✅ 密碼安全儲存，無明文洩漏
- ✅ 輸入驗證完整，防止無效資料
- ✅ 登入保護機制有效運作

#### 10.3 例外處理完整
- ✅ 自訂例外階層建立完整
- ✅ 所有可能的錯誤都有處理
- ✅ 錯誤訊息對使用者友善

#### 10.4 日誌系統有效
- ✅ 重要操作都有記錄
- ✅ 錯誤追蹤資訊完整
- ✅ 日誌格式標準化

#### 10.5 程式品質良好
- ✅ 程式碼結構清晰
- ✅ 註解與文件完整
- ✅ 符合 Python 最佳實踐

---

## 💡 開發建議

### 11. 實作提示

#### 11.1 開發順序建議
1. 🏗️ **建立基礎架構** - 例外類別、日誌系統
2. 🔍 **實作驗證模組** - 輸入驗證、安全檢查
3. 👤 **開發使用者管理** - 註冊、登入功能
4. 🖥️ **整合使用者介面** - 選單系統、互動流程
5. 🧪 **測試與除錯** - 功能測試、邊界測試

#### 11.2 常見陷阱
- ❌ 密碼以明文儲存
- ❌ 例外訊息洩漏敏感資訊
- ❌ 沒有適當的輸入驗證
- ❌ 日誌記錄不完整
- ❌ 錯誤處理不夠細緻

#### 11.3 最佳實踐
- ✅ 使用類型提示 (Type Hints)
- ✅ 遵循單一職責原則
- ✅ 避免硬編碼常數
- ✅ 使用適當的設計模式
- ✅ 保持程式碼簡潔易讀

---

## 📚 參考資源

### 12. 學習資源

#### 12.1 Python 官方文件
- [Errors and Exceptions](https://docs.python.org/3/tutorial/errors.html)
- [Logging HOWTO](https://docs.python.org/3/howto/logging.html)
- [hashlib — Secure hashes and message digests](https://docs.python.org/3/library/hashlib.html)

#### 12.2 安全性參考
- [OWASP Top 10](https://owasp.org/www-project-top-ten/)
- [Python Security](https://python-security.readthedocs.io/)
- [NIST Password Guidelines](https://pages.nist.gov/800-63-3/)

#### 12.3 最佳實踐
- [PEP 8 – Style Guide for Python Code](https://pep8.org/)
- [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html)
- [Real Python - Exception Handling](https://realpython.com/python-exceptions/)

---

**祝您開發順利！建立安全可靠的使用者註冊系統！** 🚀